# SAP Cloud ALM Process Hierarchy API Demo

This notebook contains examples of SAP Business Hub API calls for SAP Cloud ALM for implementation. The specific APIs shown below are for Process Hierarchy.

The API information and specification is available here:

  - https://api.sap.com/package/SAPCloudALM/rest - SAP Cloud ALM
  - https://api.sap.com/api/CALM_PH/overview - SAP Cloud ALM Process Hierarchy

Please note the license and other terms and conditions contained in this notebook's repository: https://github.com/SAP-samples/cloud-alm-api-examples.

## Python Dependencies Required

In order to run the samples in this notebook, install the following dependencies:

  - Jupyter integration in Visual Studio Code: https://code.visualstudio.com/docs/python/jupyter-support
  - Python 3, a recent version, is Required. Python 3.9 was used here
  - Requests - for handling HTTP GET/POST/PATCH/DELETE Requests - https://requests.readthedocs.io/en/latest/user/install/#install
  - Requests-OAuthlib - for authentication with requests - https://requests-oauthlib.readthedocs.io/en/latest/index.html#installation
  - Pandas - Python data analysis - https://pandas.pydata.org/docs/getting_started/install.html

## APIs called

API for Process Hierarchy: https://\<tenanant url\>/api/calm-processhierarchy/v1

***
# Authentication information

You must create a python module file called apidata.py and put the information specific to your tenant there. This includes:

  - OAuth2 client ID and secret
  - Token url
  - Base URL for API calls
  
Get client ID and secret variables from an external module: this information is sensitive.

These items can be retrieved from the SAP Business Technology Platform (SAP BTP) Cockpit.

## Format of module apidata.py for import

```python
service_instance_client_id = r'get your client ID from SAP BTP Cockpit'
service_instance_client_secret = r'get your client secret from SAP BTP Cockpit'
token_url = 'your token url'
base_url = 'your base url'
```

In [ ]:
# Import credentials from apidata.py
import apidata as ad

client_id = ad.ptm_all_client_id
client_secret = ad.ptm_all_client_secret
token_url = ad.token_url
base_url = ad.base_url

default_project = '11111111-1111-1111-1111-111111111111'

* * *
# Get token for authentication
Call OAuth token API with credential information. Add the token to variable which is used as header in all subsequent requests.

See Requests-OAuthlib documentation for Backend Application Flow:

* https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow

In [ ]:
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import pandas as pd

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
        client_secret=client_secret)

# Prepare the header data for all subsequent requests
headers = {'Authorization': 'Bearer ' + token['access_token']}

# prepare the header for patch requests
patchUpdateHeader = {
    'Authorization': 'Bearer ' + token['access_token'],
    'Content-Type': 'application/json'
}

* * * 
# CREATE two root hierarchy nodes

Expected response: "201 Created"

In [ ]:
hierarchyNodeIds = []
hierarchyNodeData = [
    {
        "title": "Parent Hierarchy Node 1",
        "description": "This is a new hierarchy node"
    },
    {
        "title": "Parent Hierarchy Node 2",
        "description": "This is another root hierarchy node"
    }
]

for nodeData in hierarchyNodeData:
    response = requests.post(base_url + '/HierarchyNodes', headers=headers, json=nodeData)

    if response.status_code == 201:
        print('Hierarchy node created successfully.')
        hierarchyNodeIds.append(response.json()['uuid'])
        print('Hierarchy Node ID:', hierarchyNodeIds[-1])
    else:
        print('Failed to create hierarchy node. Response:', response.text)

* * * 
# CREATE two child hierarchy nodes

Expected response: "201 Created"

In [ ]:
childHierarchyNodeData = [
    {
        "title": "Child Hierarchy Node 1",
        "description": "This is a child hierarchy node",
        "parentNodeUUID": hierarchyNodeIds[0]
    },
    {
        "title": "Child Hierarchy Node 2",
        "description": "This is another child hierarchy node",
        "parentNodeUUID": hierarchyNodeIds[0],
        "toExternalReferences": [
            {
                "name": "My Third Party System",
                "externalReferenceId": "ec44740b-7001-4ea1-a61a-4ecae614431f",
                "url": "https://my.thirdpartysystem/hierarchy/ec44740b-7001-4ea1-a61a-4ecae614431f"
            }
        ]
    }
]

for childNodeData in childHierarchyNodeData:
    response = requests.post(base_url + '/HierarchyNodes', headers=headers, json=childNodeData)

    if response.status_code == 201:
        print('Child hierarchy node created successfully.')
        hierarchyNodeIds.append(response.json()['uuid'])
        print('Hierarchy Node ID:', hierarchyNodeIds[-1])
    else:
        print('Failed to create child hierarchy node. Response:', response.text)

* * * 
# GET count of all process hierachy nodes

Expected response: "200 OK"

In [ ]:
response = requests.get(base_url + '/HierarchyNodes?$count=true', headers=headers)

if response.status_code == 200:
    print(response.status_code, response.reason)
    print('Number of hierarchy nodes: ', response.json()["@count"])
else:
    print('Failed to fetch number of hierarchy nodes. Response:', response.text)

* * * 
# GET a list of process hierarchy nodes

Expected response: "200 OK"

In [ ]:
response = requests.get(base_url + '/HierarchyNodes', headers=headers)

if response.status_code == 200:
    print(response.status_code, response.reason)
    print('Hierarchy Nodes: ', response.json()["value"])
else:
    print('Failed to fetch hierarchy nodes. Response:', response.text)

***

# Update a hierarchy node

Expected response: "200 OK"

In [ ]:
hierachyNodeUpdateData = {
    "title": "Updated Child Hierarchy Node 2",
    "toExternalReferences": [
        {
            "name": "My Renamed Third Party System",
            "externalReferenceId": "ec44740b-7001-4ea1-a61a-4ecae614431f",
            "url": "https://my.renamed.thirdpartysystem/hierarchy/ec44740b-7001-4ea1-a61a-4ecae614431f"
        },
        {
            "name": "My New Third Party System",
            "externalReferenceId": "ec44740b-7001-4ea1-a61a-4ecae614431f",
            "url": "https://my.new.thirdpartysystem/hierarchy/ec44740b-7001-4ea1-a61a-4ecae614431f"
        }
    ]
}

response = requests.patch(base_url + '/HierarchyNodes/' + hierarchyNodeIds[3], headers=patchUpdateHeader, json=hierachyNodeUpdateData)

if response.status_code == 200:
    print(response.status_code, response.reason)
    print('Updated hierarchy node: ', response.json())
else:
    print('Failed to update hierarchy nodes. Response:', response.text)

***

# Reach a hierarchy node by External Reference

Expected response: "200 OK"

In [ ]:
externalReferenceId = "ec44740b-7001-4ea1-a61a-4ecae614431f"
externalReferenceName = "My Renamed Third Party System"

filterQuery = "toExternalReferences/any(er:er/name eq '" + externalReferenceName + "' and er/externalReferenceId eq '" + externalReferenceId + "')"
expandQuery = "toExternalReferences"

response = requests.get(base_url + '/HierarchyNodes?$expand=' + expandQuery + "&$filter=" + filterQuery, headers=headers)

if response.status_code == 200:
    print(response.status_code, response.reason)
    print('Hierarchy node: ', response.json()["value"])
else:
    print('Failed to reach hierarchy node. Response:', response.text)

***

# Change the sequence of a hierarchy node

Expected response: "200 OK"

In [ ]:
hierachyNodeUpdateData = {
    "sequence": 3
}

response = requests.patch(base_url + '/HierarchyNodes/' + hierarchyNodeIds[3], headers=patchUpdateHeader, json=hierachyNodeUpdateData)

if response.status_code == 200:
    print(response.status_code, response.reason)
    print('Updated hierarchy node: ', response.json())
else:
    print('Failed to update hierarchy nodes. Response:', response.text)

***

# Move a child node to a different parent

Expected response: "200 OK"

In [ ]:
hierachyNodeUpdateData = {
    "parentNodeUUID": hierarchyNodeIds[1]
}

response = requests.patch(base_url + '/HierarchyNodes/' + hierarchyNodeIds[2], headers=patchUpdateHeader, json=hierachyNodeUpdateData)

if response.status_code == 200:
    print(response.status_code, response.reason)
    print('Updated hierarchy node: ', response.json())
else:
    print('Failed to update hierarchy nodes. Response:', response.text)

***

# Delete a root hierarchy node

Expected response: "204 OK"

In [ ]:
response = requests.delete(base_url + '/HierarchyNodes/' + hierarchyNodeIds[1], headers=headers)


if response.status_code == 204:
    print(response.status_code, response.reason)
    print('Deleted hierarchy!')
else:
    print('Failed to delete hierarchy node. Response:', response.text)